# Import Libraries

In [4]:
import sys
import os
import sqlite3
import warnings
import json

import pandas as pd
import numpy as np

import optuna
import shap
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier # Changed from ExtraTreesClassifier
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, classification_report, confusion_matrix
from sklearn.feature_selection import RFECV


warnings.filterwarnings('ignore')
np.random.seed(42)

# Load Database

In [5]:
sys.path.append('/home/pooya/w/DroughtMonitoringIran/')

DATABASE_PATH = "./database/database.db"

RESULTS_DIR = 'results'

os.makedirs(RESULTS_DIR, exist_ok=True)

conn = sqlite3.connect(DATABASE_PATH)

data = pd.read_sql(sql='SELECT * FROM data', con=conn)

conn.close()

In [ ]:
def run(di, di_scale, selected_stations, stations_group_name, start_date, end_date):
    
    group_dir = os.path.join(RESULTS_DIR, stations_group_name)
    os.makedirs(group_dir, exist_ok=True)
    
    scale_dir = os.path.join(group_dir, f'{di}_{di_scale}')
    os.makedirs(scale_dir, exist_ok=True)
    
    # Select Columns
    selected_columns = [
        'Station_Name', 'Station_ID',
        'Station_Latitude', 'Station_Longitude', 'Station_Elevation',
        'Date',
        f'{di}_{di_scale}',
        f'GPM_{di}_{di_scale}',
        f'ERA5_{di}_{di_scale}',
        'ERA5_Precipitation',
        'GPM_Precipitation',
        'PET_MOD16A2GF',
        'NDVI', 'EVI',
        'LSTDay', 'LSTNight', 'LST',
        'PCI_GPM', 'PCI_ERA5',
        'VCI', 'TCI', 'VHI',
        'CI_GPM', 'CI_ERA5',
    ]
    
    df = data\
    .filter(items=selected_columns)\
        .query("Station_Name in @selected_stations and Date >= @start_date and Date < @end_date")

    # Date, Year, Month
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m')
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    # SPI and SPEI Class
    df[f'{di}_{di_scale}_Class'] = pd.cut(df[f'{di}_{di_scale}'], bins=[-10, -2, -1.5, -1, 1, 1.5, 2, 10], labels=['ED', 'SD', 'MD', 'NN', 'MW', 'VW', 'EW'])

    # Month Sin & Cos
    df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
    df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
            
    # LST Diff
    df['LST_Diff'] = df['LSTDay'] - df['LSTNight']

    # Convert to Category
    df['Station_ID'] = df['Station_ID'].astype('category')
    df['Year'] = df['Year'].astype('category')
    df['Month'] = df['Month'].astype('category')
    df[f'{di}_{di_scale}_Class'] = df[f'{di}_{di_scale}_Class'].astype('category')
    
    df.dropna(inplace=True)
    df.sort_values(by=['Station_ID', 'Year', 'Month'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    selected_columns_lag_roll = [
        'ERA5_Precipitation',
        'GPM_Precipitation',
        f'GPM_{di}_{di_scale}',
        f'ERA5_{di}_{di_scale}',
        'PET_MOD16A2GF',
        'NDVI', 'EVI',
        'LSTDay', 'LSTNight', 'LST',
        'PCI_GPM', 'PCI_ERA5',
        'VCI', 'TCI', 'VHI',
        'CI_GPM', 'CI_ERA5',
    ]

    # Add Lag
    for lag in range(1, 4):
        for col in selected_columns_lag_roll:
            df[f'{col}_lag_{lag}'] = df.groupby('Station_ID', observed=False)[col].shift(lag)


    # # Add Mean and Std Roll
    # for r in [3, 6]:
    #     for col in selected_columns_lag_roll:
    #         df[f'{col}_roll_mean_{r}'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=r, min_periods=1).mean())
    #         df[f'{col}_roll_std_{r}'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=r, min_periods=1).std())
    
    
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # Define columns to exclude from features
    EXCLUDE_COLS = ['Station_Name', 'Station_ID', 'Date', f'{di}_{di_scale}_Class', f'{di}_{di_scale}', 'Year', 'Month']
    FEATURES = [col for col in df.columns if col not in EXCLUDE_COLS]

    # Station-wise standardization
    df_scaled = df.copy()
    for station in df_scaled['Station_Name'].unique():
        station_mask = df_scaled['Station_Name'] == station
        scaler = StandardScaler()
        df_scaled.loc[station_mask, FEATURES] = scaler.fit_transform(df_scaled.loc[station_mask, FEATURES])


    class_counts = df_scaled[f'{di}_{di_scale}_Class'].value_counts()
    classes_to_remove = class_counts[class_counts < 2].index

    if not classes_to_remove.empty:
        print(f"Removing classes with fewer than 2 samples: {classes_to_remove.tolist()}")
        
        # Filter the dataframe to exclude these rare classes
        df_filtered = df_scaled[~df_scaled[f'{di}_{di_scale}_Class'].isin(classes_to_remove)].copy()
        
        # Redefine X and y from the filtered dataframe
        X = df_filtered[FEATURES]
        y = df_filtered[f'{di}_{di_scale}_Class']
        
        print(f"Data shape after removing rare classes: {X.shape}")
        print("\n--- Final Class Distribution ---")
        print(y.value_counts())
    else:
        print("No classes with fewer than 2 samples found.")
        X = df_scaled[FEATURES]
        y = df_scaled[f'{di}_{di_scale}_Class']

    # Encode the target variable
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y_encoded,
        test_size=0.30,
        random_state=42,
        stratify=y_encoded
    )

    print(f"Target Classes: {le.classes_}")
    print(f"Training set shape: {X_train.shape}")
    print(f"Testing set shape: {X_test.shape}")


    # The estimator that will be used by RFECV
    estimator = RandomForestClassifier(
        random_state=42,
        n_jobs=-1
    )

    # The RFECV object
    rfecv = RFECV(
        estimator=estimator,
        step=1,
        cv=StratifiedKFold(5),
        scoring='f1_macro',
        n_jobs=-1
    )

    # Fit RFECV on the training data
    rfecv.fit(X_train, y_train)

    print(f"Optimal number of features found: {rfecv.n_features_}")

    # Get the final selected features
    final_features = X_train.columns[rfecv.support_]
    print(f"Selected features: {final_features.tolist()}")
    
    features_path = os.path.join(scale_dir, 'selected_features.txt')
    with open(features_path, 'w') as f:
        for feature in final_features:
            f.write(f"{feature}\n")
    print(f"Selected features saved to {features_path}")

    # --- Plot Feature Importances ---
    # The `rfecv.estimator_` attribute is the model trained on the full set of features
    # during the last step of the cross-validation process. We can use its feature importances.
    importances = rfecv.estimator_.feature_importances_
    feature_importance_df = pd.DataFrame({
        'Feature': final_features,
        'Importance': importances
    }).sort_values(by='Importance', ascending=False)

    # Plot only the selected features
    selected_feature_importances = feature_importance_df[feature_importance_df['Feature'].isin(final_features)]

    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=selected_feature_importances)
    plt.title('Feature Importances for Selected Features (from RFECV)')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.tight_layout()
    fig_path = os.path.join(scale_dir, 'feature_importances.png')
    plt.savefig(fig_path)
    plt.close()


    # Update training and testing sets with selected features
    X_train_final = X_train[final_features]
    X_test_final = X_test[final_features]
    
    
    def objective(trial):
        """Define the objective function for Optuna to optimize."""
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'max_depth': trial.suggest_int('max_depth', 5, 50),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
            'max_features': trial.suggest_float('max_features', 0.1, 1.0),
            'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
            'random_state': 42,
            'n_jobs': -1
        }
        
        model = RandomForestClassifier(**params)
        
        # Stratified K-Fold Cross-Validation
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        f1_scores = []
        
        for train_idx, val_idx in skf.split(X_train_final, y_train):
            X_train_fold, X_val_fold = X_train_final.iloc[train_idx], X_train_final.iloc[val_idx]
            y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
            
            model.fit(X_train_fold, y_train_fold)
            preds = model.predict(X_val_fold)
            f1_scores.append(f1_score(y_val_fold, preds, average='macro'))
            
        return np.mean(f1_scores)

    # Create a study object and optimize
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=50, timeout=600) # 50 trials or 10 minutes

    # Save best parameters
    best_params = study.best_params 
    params_path = os.path.join(scale_dir, 'best_params.json')
    with open(params_path, 'w') as f:
        json.dump(best_params, f, indent=4)
    print(f"Best parameters saved to {params_path}")

    print(f"Best trial found: {study.best_trial.number}")
    print(f"Best Macro-F1 Score: {study.best_value}")
    print("Best Hyperparameters:")
    print(study.best_params)



    # Get best hyperparameters
    best_params = study.best_params
    best_params['random_state'] = 42
    best_params['n_jobs'] = -1

    # Train the final model
    final_model = RandomForestClassifier(**best_params)
    final_model.fit(X_train_final, y_train)

    # Make predictions on the test set
    y_pred = final_model.predict(X_test_final)

    # Save evaluation metrics
    report = classification_report(y_test, y_pred, target_names=le.classes_)
    accuracy = accuracy_score(y_test, y_pred)
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    kappa = cohen_kappa_score(y_test, y_pred)
    
    metrics_path = os.path.join(scale_dir, 'evaluation_metrics.txt')
    with open(metrics_path, 'w') as f:
        f.write(f"Overall Accuracy: {accuracy:.4f}\n")
        f.write(f"Macro-F1 Score: {macro_f1:.4f}\n")
        f.write(f"Cohen's Kappa: {kappa:.4f}\n\n")
        f.write("Classification Report:\n")
        f.write(report)
    print(f"Evaluation metrics saved to {metrics_path}")

    print(f"Overall Accuracy: {accuracy:.4f}")
    print(f"Macro-F1 Score: {macro_f1:.4f}")
    print(f"Cohen's Kappa: {kappa:.4f}")

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))


    # --- Plot Confusion Matrix ---
    print("\n--- Plotting Confusion Matrix ---")
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title('Confusion Matrix')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    fig_path = os.path.join(scale_dir, 'confusion_matrix.png')
    plt.savefig(fig_path)
    plt.close()
        

    # Create SHAP explainer for the final model
    # Using a subset of the training data for the background is a common practice for performance
    explainer = shap.TreeExplainer(final_model, X_train_final.sample(100, random_state=42))
    shap_values = explainer.shap_values(X_test_final)

    try:
        # SHAP Summary Plot (Bar)
        print("\nGenerating SHAP Summary Bar Plot...")
        plt.figure()
        shap.summary_plot(shap_values, X_test_final, plot_type="bar", class_names=le.classes_, show=False)
        plt.title("SHAP Summary Plot (Global Feature Importance)")
        plt.tight_layout()
        fig_path = os.path.join(scale_dir, 'shap_summary_bar.png')
        plt.savefig(fig_path, bbox_inches='tight')
        plt.close()
    except:
        print("An exception occurred: Generating SHAP Summary Bar Plot...")
        
    # try:
    #     # SHAP Summary Plot (Beeswarm)
    #     print("\nGenerating SHAP Summary Beeswarm Plot...")
    #     plt.figure()
    #     shap.summary_plot(shap_values, X_test_final, class_names=le.classes_, show=False,  plot_type="violin")
    #     plt.title("SHAP Summary Plot (Beeswarm)")
    #     plt.tight_layout()
    #     fig_path = os.path.join(scale_dir, 'shap_summary_beeswarm.png')
    #     plt.savefig(fig_path, bbox_inches='tight')
    #     plt.close()
    # except:
    #     print("An exception occurred: SHAP Summary Plot (Beeswarm)")
    
    return None



for di in ['SPI', 'SPEI']:
    for di_scale in [1, 3, 6, 9, 12, 18, 24]:
        for stations_group_name in ['ALL', 'C1', 'C2', 'C3']: 

            if stations_group_name == 'ALL':
                selected_stations = ['Ramsar', 'Nowshahr', 'Siahbisheh', 'Kiyasar', 'Kojur', 'Baladeh', 'Alasht', 'Babolsar', 'Gharakhil', 'Sari', 'Sari (dasht-e-naz airport)', 'Galugah', 'Bandar-e-amirabad', 'Amol', 'Polsefid']
            elif stations_group_name == 'C3':
                selected_stations = ['Ramsar', 'Nowshahr']
            elif stations_group_name == 'C2':
                selected_stations = ['Siahbisheh', 'Kiyasar', 'Kojur', 'Baladeh', 'Alasht']
            elif stations_group_name == 'C1':
                selected_stations = ['Babolsar', 'Gharakhil', 'Sari', 'Sari (dasht-e-naz airport)', 'Galugah', 'Bandar-e-amirabad', 'Amol', 'Polsefid']
            else:
                selected_stations = ['Ramsar', 'Nowshahr', 'Siahbisheh', 'Kiyasar', 'Kojur', 'Baladeh', 'Alasht', 'Babolsar', 'Gharakhil', 'Sari', 'Sari (dasht-e-naz airport)', 'Galugah', 'Bandar-e-amirabad', 'Amol', 'Polsefid']

            ex_dir = os.path.join(RESULTS_DIR, stations_group_name, f'{di}_{di_scale}')
            if os.path.exists(ex_dir):
                continue 
            
            run(
                di=di,
                di_scale=di_scale,
                selected_stations=selected_stations,
                stations_group_name=stations_group_name,
                start_date='2006-09', 
                end_date='2023-10'
            )

No classes with fewer than 2 samples found.
Target Classes: ['ED' 'EW' 'MD' 'MW' 'NN' 'SD' 'VW']
Training set shape: (2121, 74)
Testing set shape: (909, 74)
Optimal number of features found: 40
Selected features: ['GPM_SPI_1', 'ERA5_SPI_1', 'ERA5_Precipitation', 'GPM_Precipitation', 'PET_MOD16A2GF', 'NDVI', 'LSTDay', 'LST', 'PCI_GPM', 'PCI_ERA5', 'TCI', 'VHI', 'CI_GPM', 'CI_ERA5', 'LST_Diff', 'ERA5_Precipitation_lag_1', 'GPM_SPI_1_lag_1', 'PET_MOD16A2GF_lag_1', 'NDVI_lag_1', 'EVI_lag_1', 'PCI_GPM_lag_1', 'PCI_ERA5_lag_1', 'TCI_lag_1', 'VHI_lag_1', 'CI_GPM_lag_1', 'ERA5_Precipitation_lag_2', 'GPM_SPI_1_lag_2', 'ERA5_SPI_1_lag_2', 'EVI_lag_2', 'LSTDay_lag_2', 'LSTNight_lag_2', 'VCI_lag_2', 'TCI_lag_2', 'ERA5_Precipitation_lag_3', 'GPM_Precipitation_lag_3', 'EVI_lag_3', 'LSTNight_lag_3', 'PCI_GPM_lag_3', 'TCI_lag_3', 'CI_GPM_lag_3']
Selected features saved to results/ALL/SPI_1/selected_features.txt


[I 2025-08-24 02:03:15,209] A new study created in memory with name: no-name-ca7b3901-6a9d-4667-ab54-c18079dec7fa
[I 2025-08-24 02:03:18,489] Trial 0 finished with value: 0.23832738050857388 and parameters: {'n_estimators': 204, 'max_depth': 48, 'min_samples_split': 14, 'min_samples_leaf': 15, 'max_features': 0.4004911152393068, 'criterion': 'gini'}. Best is trial 0 with value: 0.23832738050857388.
[I 2025-08-24 02:03:26,344] Trial 1 finished with value: 0.2732131088224911 and parameters: {'n_estimators': 427, 'max_depth': 5, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': 0.9327846806191983, 'criterion': 'gini'}. Best is trial 1 with value: 0.2732131088224911.
[I 2025-08-24 02:03:33,392] Trial 2 finished with value: 0.26415662824950115 and parameters: {'n_estimators': 258, 'max_depth': 10, 'min_samples_split': 3, 'min_samples_leaf': 12, 'max_features': 0.46269354194336176, 'criterion': 'gini'}. Best is trial 1 with value: 0.2732131088224911.
[I 2025-08-24 02:03:39,695] 

Best parameters saved to results/ALL/SPI_1/best_params.json
Best trial found: 32
Best Macro-F1 Score: 0.32758578343297395
Best Hyperparameters:
{'n_estimators': 457, 'max_depth': 18, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 0.9382813249566467, 'criterion': 'gini'}
Evaluation metrics saved to results/ALL/SPI_1/evaluation_metrics.txt
Overall Accuracy: 0.6755
Macro-F1 Score: 0.2439
Cohen's Kappa: 0.2299

Classification Report:
              precision    recall  f1-score   support

          ED       0.29      0.10      0.15        41
          EW       0.25      0.14      0.18        21
          MD       0.11      0.04      0.05        55
          MW       0.33      0.19      0.24        97
          NN       0.75      0.95      0.83       608
          SD       0.00      0.00      0.00        27
          VW       0.34      0.20      0.25        60

    accuracy                           0.68       909
   macro avg       0.30      0.23      0.24       909
weighted

 83%|=================   | 5257/6363 [04:14<00:53]       

# Select Columns and Rows

In [ ]:
# Select Drought Index and Period
di = 'SPEI'
di_scale = 3

# Select Stations
selected_stations = ['Ramsar', 'Nowshahr', 'Siahbisheh', 'Kiyasar', 'Kojur', 'Baladeh', 'Alasht', 'Babolsar', 'Gharakhil', 'Sari', 'Sari (dasht-e-naz airport)', 'Galugah', 'Bandar-e-amirabad', 'Amol', 'Polsefid']
# selected_stations = ['Ramsar', 'Nowshahr']
# selected_stations = ['Siahbisheh', 'Kiyasar', 'Kojur', 'Baladeh', 'Alasht']
# selected_stations = ['Babolsar', 'Gharakhil', 'Sari', 'Sari (dasht-e-naz airport)', 'Galugah', 'Bandar-e-amirabad', 'Amol', 'Polsefid']

# Select Columns
selected_columns = [
    'Station_Name', 'Station_ID',
    'Station_Latitude', 'Station_Longitude', 'Station_Elevation',
    'Date',
    f'{di}_{di_scale}',
    f'GPM_{di}_{di_scale}',
    f'ERA5_{di}_{di_scale}',
    'ERA5_Precipitation',
    'GPM_Precipitation',
    'PET_MOD16A2GF',
    'NDVI', 'EVI',
    'LSTDay', 'LSTNight', 'LST',
    'PCI_GPM', 'PCI_ERA5',
    'VCI', 'TCI', 'VHI',
    'CI_GPM', 'CI_ERA5',
]

# Select Start and End Date
start_date = '2006-09'
end_date = '2023-10'

# Filter Data

In [ ]:
df = data\
    .filter(items=selected_columns)\
        .query("Station_Name in @selected_stations and Date >= @start_date and Date < @end_date")

# Add Some Columns

In [ ]:
# Date, Year, Month
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m')
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

# SPI and SPEI Class
df[f'{di}_{di_scale}_Class'] = pd.cut(df[f'{di}_{di_scale}'], bins=[-10, -2, -1.5, -1, 1, 1.5, 2, 10], labels=['ED', 'SD', 'MD', 'NN', 'MW', 'VW', 'EW'])

# Month Sin & Cos
df['Month_sin'] = np.sin(2 * np.pi * df['Month'] / 12)
df['Month_cos'] = np.cos(2 * np.pi * df['Month'] / 12)
        
# LST Diff
df['LST_Diff'] = df['LSTDay'] - df['LSTNight']

# Convert to Category
df['Station_ID'] = df['Station_ID'].astype('category')
df['Year'] = df['Year'].astype('category')
df['Month'] = df['Month'].astype('category')
df[f'{di}_{di_scale}_Class'] = df[f'{di}_{di_scale}_Class'].astype('category')

In [ ]:
df.dropna(inplace=True)
df.sort_values(by=['Station_ID', 'Year', 'Month'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
selected_columns_lag_roll = [
    'ERA5_Precipitation',
    'GPM_Precipitation',
    f'GPM_{di}_{di_scale}',
    f'ERA5_{di}_{di_scale}',
    'PET_MOD16A2GF',
    'NDVI', 'EVI',
    'LSTDay', 'LSTNight', 'LST',
    'PCI_GPM', 'PCI_ERA5',
    'VCI', 'TCI', 'VHI',
    'CI_GPM', 'CI_ERA5',
]

# Add Lag
for lag in range(1, 4):
    for col in selected_columns_lag_roll:
        df[f'{col}_lag_{lag}'] = df.groupby('Station_ID', observed=False)[col].shift(lag)


# Add Mean and Std Roll
for r in [3, 6]:
    for col in selected_columns_lag_roll:
        df[f'{col}_roll_mean_{r}'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=r, min_periods=1).mean())
        df[f'{col}_roll_std_{r}'] = df.groupby('Station_ID', observed=False)[col].transform(lambda x: x.rolling(window=r, min_periods=1).std())

# Remove all NA

In [ ]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# Station-wise Standardization

In [ ]:
# Define columns to exclude from features
EXCLUDE_COLS = ['Station_Name', 'Station_ID', 'Date', f'{di}_{di_scale}_Class', f'{di}_{di_scale}', 'Year', 'Month']
FEATURES = [col for col in df.columns if col not in EXCLUDE_COLS]

# Station-wise standardization
df_scaled = df.copy()
for station in df_scaled['Station_Name'].unique():
    station_mask = df_scaled['Station_Name'] == station
    scaler = StandardScaler()
    df_scaled.loc[station_mask, FEATURES] = scaler.fit_transform(df_scaled.loc[station_mask, FEATURES])


class_counts = df_scaled[f'{di}_{di_scale}_Class'].value_counts()
classes_to_remove = class_counts[class_counts < 2].index

if not classes_to_remove.empty:
    print(f"Removing classes with fewer than 2 samples: {classes_to_remove.tolist()}")
    
    # Filter the dataframe to exclude these rare classes
    df_filtered = df_scaled[~df_scaled[f'{di}_{di_scale}_Class'].isin(classes_to_remove)].copy()
    
    # Redefine X and y from the filtered dataframe
    X = df_filtered[FEATURES]
    y = df_filtered[f'{di}_{di_scale}_Class']
    
    print(f"Data shape after removing rare classes: {X.shape}")
    print("\n--- Final Class Distribution ---")
    print(y.value_counts())
else:
    print("No classes with fewer than 2 samples found.")
    X = df_scaled[FEATURES]
    y = df_scaled[f'{di}_{di_scale}_Class']

# Encode the target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=0.30,
    random_state=42,
    stratify=y_encoded
)

print(f"Target Classes: {le.classes_}")
print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

# Feature Selection with RFECV

In [ ]:
# The estimator that will be used by RFECV
estimator = RandomForestClassifier(
    random_state=42,
    n_jobs=-1
)

# The RFECV object
rfecv = RFECV(
    estimator=estimator,
    step=1,
    cv=StratifiedKFold(5),
    scoring='f1_macro',
    n_jobs=-1
)

# Fit RFECV on the training data
rfecv.fit(X_train, y_train)

print(f"Optimal number of features found: {rfecv.n_features_}")

# Get the final selected features
final_features = X_train.columns[rfecv.support_]
print(f"Selected features: {final_features.tolist()}")

# --- Plot Feature Importances ---
# The `rfecv.estimator_` attribute is the model trained on the full set of features
# during the last step of the cross-validation process. We can use its feature importances.
importances = rfecv.estimator_.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': final_features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Plot only the selected features
selected_feature_importances = feature_importance_df[feature_importance_df['Feature'].isin(final_features)]

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=selected_feature_importances)
plt.title('Feature Importances for Selected Features (from RFECV)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()


# Update training and testing sets with selected features
X_train_final = X_train[final_features]
X_test_final = X_test[final_features]

# Hyperparameter Tuning with Optuna

In [ ]:
def objective(trial):
    """Define the objective function for Optuna to optimize."""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
        'random_state': 42,
        'n_jobs': -1
    }
    
    model = RandomForestClassifier(**params)
    
    # Stratified K-Fold Cross-Validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    f1_scores = []
    
    for train_idx, val_idx in skf.split(X_train_final, y_train):
        X_train_fold, X_val_fold = X_train_final.iloc[train_idx], X_train_final.iloc[val_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
        
        model.fit(X_train_fold, y_train_fold)
        preds = model.predict(X_val_fold)
        f1_scores.append(f1_score(y_val_fold, preds, average='macro'))
        
    return np.mean(f1_scores)

# Create a study object and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, timeout=600) # 50 trials or 10 minutes

print(f"Best trial found: {study.best_trial.number}")
print(f"Best Macro-F1 Score: {study.best_value}")
print("Best Hyperparameters:")
print(study.best_params)


# Model Training & Evaluation

In [ ]:
# Get best hyperparameters
best_params = study.best_params
best_params['random_state'] = 42
best_params['n_jobs'] = -1

# Train the final model
final_model = RandomForestClassifier(**best_params)
final_model.fit(X_train_final, y_train)

# Make predictions on the test set
y_pred = final_model.predict(X_test_final)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
macro_f1 = f1_score(y_test, y_pred, average='macro')
kappa = cohen_kappa_score(y_test, y_pred)

print(f"Overall Accuracy: {accuracy:.4f}")
print(f"Macro-F1 Score: {macro_f1:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


# --- Plot Confusion Matrix ---
print("\n--- Plotting Confusion Matrix ---")
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# Explainability with SHAP

In [ ]:
# Create SHAP explainer for the final model
# Using a subset of the training data for the background is a common practice for performance
explainer = shap.TreeExplainer(final_model, X_train_final.sample(100, random_state=42))
shap_values = explainer.shap_values(X_test_final)

# SHAP Summary Plot (Bar)
print("\nGenerating SHAP Summary Bar Plot...")
plt.figure()
shap.summary_plot(shap_values, X_test_final, plot_type="bar", class_names=le.classes_, show=False)
plt.title("SHAP Summary Plot (Global Feature Importance)")
plt.tight_layout()
plt.show()

# SHAP Summary Plot (Beeswarm)
print("\nGenerating SHAP Summary Beeswarm Plot...")
plt.figure()
shap.summary_plot(shap_values, X_test_final, class_names=le.classes_, show=False,  plot_type="violin")
plt.title("SHAP Summary Plot (Beeswarm)")
plt.tight_layout()
plt.show()